In [1]:
# notebooks/04_BERT_Analysis.ipynb
# Importar las bibliotecas necesarias
import os
import logging
import sys
import pandas as pd

# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils.file_utils import read_excel_file, save_data_to_excel
from utils.advanced_ner_utils import (
    perform_ner_with_bert,
    process_sentences_and_extract_entities,
)

from config import XLSX_DIRECTORY
from utils.ner_utils import cluster_entities
from transformers import BertTokenizer, BertForTokenClassification

# Carga de modelos y tokenizers
bert_model_name = "mrm8488/bert-spanish-cased-finetuned-ner"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertForTokenClassification.from_pretrained(bert_model_name)


def main():
    # Cargar los datos de las oraciones desde el archivo Excel
    logging.info("Cargando datos de oraciones desde Excel...")
    sentence_data = read_excel_file(os.path.join(XLSX_DIRECTORY, "sentence.xlsx"))
    # sentence_data = read_excel_file(os.path.join(XLSX_DIRECTORY, "sentence_2.xlsx"))

    # Realizar NER con BERT
    logging.info("Realizando NER con BERT...")
    bert_entities_df = process_sentences_and_extract_entities(
        sentence_data, bert_tokenizer, bert_model, perform_ner_with_bert, "bert"
    )

    # Convertir DataFrame de entidades a lista de diccionarios para el clustering
    bert_entities_df = bert_entities_df.to_dict("records")

    # Opcional: Agrupar entidades similares con K-means
    logging.info("Agrupando entidades similares...")
    bert_entities_df = cluster_entities(bert_entities_df, num_clusters=10)

    # Guardar los datos de entidades BERT en un nuevo archivo Excel
    logging.info("Guardando datos de entidades BERT en Excel...")
    save_data_to_excel(bert_entities_df, XLSX_DIRECTORY, "bert_entity.xlsx")


if __name__ == "__main__":
    main()

2024-01-21 17:35:39,242 - INFO - loading weights file pytorch_model.bin from cache at C:\Users\User\.cache\huggingface\hub\models--mrm8488--bert-spanish-cased-finetuned-ner\snapshots\b11721d41d9e948da32fcdabeeef4fb0f3ebcdf7\pytorch_model.bin
2024-01-21 17:35:39,714 - WARNING - Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2024-01-21 17:35:39,715 - INFO - All th

Procesando: 0/2208 oraciones (0.00%)
Procesando: 220/2208 oraciones (9.96%)
Procesando: 440/2208 oraciones (19.93%)

Entidad 'San  de Miraflores' no encontrada en el texto.
Contexto: primer puesto de su promoción [ENTIDAD] está por culminar sus estudios
Oración: Abigail, de 17 años está por culminar sus estudios en la escuela César Vallejo, en San Juan de Miraflores; ella es brigadier general y primer puesto de su promoción.
Omitiendo la adición de la entidad 'San  de Miraflores' ya que no se encontró en el texto.

Entidad 'San  de Miraflores' no encontrada en el texto.
Contexto: en San Juan de Miraflores [ENTIDAD] la intervención de Essalud,
Oración: A esto debe sumarse la intervención de Essalud, que construirá el Instituto de Medicina Deportiva y Rehabilitación en San Juan de Miraflores.
Omitiendo la adición de la entidad 'San  de Miraflores' ya que no se encontró en el texto.
Procesando: 660/2208 oraciones (29.89%)
Procesando: 880/2208 oraciones (39.86%)

Entidad 'UD E  SOCIAL' no 

2024-01-21 17:40:36,511 - INFO - Agrupando entidades similares...



Entidad 'lace de 500  Celendín' no encontrada en el texto.
Contexto: en 54 millones de dólares [ENTIDAD] destacan el Enlace de 500
Oración: Entre esos proyectos destacan el Enlace de 500 kilovatios Huánuco-Tocache-Celendín-Trujillo, que se ejecutará con una inversión estimada de 486 millones de dólares; el Enlace de 500 kilovatios Celendín-Piura, que demanda una inversión estimada de 234 millones de dólares; y, el Enlace 220 kilovatios Ica-Poroma, que significará una inversión estimada en 54 millones de dólares.
Omitiendo la adición de la entidad 'lace de 500  Celendín' ya que no se encontró en el texto.
Procesamiento completado.


d:\TFM_Project\PLN_Project\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
2024-01-21 17:40:37,046 - INFO - Guardando datos de entidades BERT en Excel...
2024-01-21 17:40:37,606 - INFO - DataFrame guardado en d:\TFM_Project\PLN_Project\data\xlsx\bert_entity.xlsx


In [ ]:
# pip install transformers
# pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu111

In [3]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

# Cargar el modelo y el tokenizador
bert_model_name = "mrm8488/bert-spanish-cased-finetuned-ner"
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
model = BertForTokenClassification.from_pretrained(bert_model_name)

# Texto a analizar
text = "Yo siempre digo, lo hablo mucho con los gobernadores, acá está el Gobernador de Tucumán, y lo hablo con muchos intendentes, algunos los veo por allí, lo veo a Lucas, lo veo a Andrés, lo veo a Julio, y lo hablo con muchos de ellos."

# Codificar el texto
inputs = tokenizer.encode(text, return_tensors="pt")

# Predicciones del modelo
with torch.no_grad():
    outputs = model(inputs)[0]
predictions = torch.argmax(outputs, dim=2)

# Función para reconstruir las entidades a partir de subtokens
def reconstruct_entities(tokens, predictions, tokenizer, model):
    new_tokens, new_labels = [], []
    for token, prediction in zip(tokens, predictions[0]):
        token_str = tokenizer.decode([token], skip_special_tokens=False)
        label = model.config.id2label[prediction.item()]

        # Reconstruir la entidad, incluyendo [UNK] si está rodeado por subtokens de la misma entidad
        if new_labels and label.startswith("I-") and new_labels[-1].endswith(label.split("-")[-1]):
            if token_str != tokenizer.unk_token or (new_tokens[-1] != tokenizer.unk_token and token_str == tokenizer.unk_token):
                new_tokens[-1] = new_tokens[-1] + " " + token_str
            else:
                new_tokens[-1] += token_str
        else:
            new_tokens.append(token_str)
            new_labels.append(label)

    return new_tokens, new_labels


# Reconstruir entidades
new_tokens, new_labels = reconstruct_entities(inputs[0], predictions, tokenizer, model)

# Imprimir tokens y sus etiquetas
for token, label in zip(new_tokens, new_labels):
    print(f"{token}: {label}")
print(tokenizer.tokenize(text))



2024-01-18 18:12:09,174 - INFO - loading weights file pytorch_model.bin from cache at C:\Users\User\.cache\huggingface\hub\models--mrm8488--bert-spanish-cased-finetuned-ner\snapshots\b11721d41d9e948da32fcdabeeef4fb0f3ebcdf7\pytorch_model.bin
2024-01-18 18:12:09,567 - WARNING - Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2024-01-18 18:12:09,570 - INFO - All th

[CLS]: O
Yo: O
siempre: O
digo: O
,: O
lo: O
hablo: O
mucho: O
con: O
los: O
gobernadores: O
,: O
acá: O
está: O
el: O
Gobernador de Tucumán: B-PER
,: O
y: O
lo: O
hablo: O
con: O
muchos: O
inten: O
##dentes: O
,: O
algunos: O
los: O
veo: O
por: O
allí: O
,: O
lo: O
veo: O
a: O
Lucas: B-PER
,: O
lo: O
veo: O
a: O
Andrés: B-PER
,: O
lo: O
veo: O
a: O
[UNK]: B-PER
,: O
y: O
lo: O
hablo: O
con: O
muchos: O
de: O
ellos: O
.: O
[SEP]: O
['Yo', 'siempre', 'digo', ',', 'lo', 'hablo', 'mucho', 'con', 'los', 'gobernadores', ',', 'acá', 'está', 'el', 'Gobernador', 'de', 'Tucumán', ',', 'y', 'lo', 'hablo', 'con', 'muchos', 'inten', '##dentes', ',', 'algunos', 'los', 'veo', 'por', 'allí', ',', 'lo', 'veo', 'a', 'Lucas', ',', 'lo', 'veo', 'a', 'Andrés', ',', 'lo', 'veo', 'a', '[UNK]', ',', 'y', 'lo', 'hablo', 'con', 'muchos', 'de', 'ellos', '.']
